In [97]:
import cv2
import mediapipe as mp
import numpy as np

from pygame import mixer



# Inicializar MediaPipe Pose y la cámara
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Capturar video desde la cámara
cap = cv2.VideoCapture(1)
mixer.init()
ruta_mp3 = "C:/Users/fierr/OneDrive/Documents/proyectos/pose/sonido.mp3"
mixer.music.load(ruta_mp3)
mixer.music.set_volume(0.7)
mixer.music.play()

puntos_max=50
lista_puntos_izq=[]
lista_puntos_der=[]
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error al capturar el video")
        break

    # Convertir la imagen a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar la imagen con MediaPipe Pose
    results = pose.process(frame_rgb)

    # Si se detectan puntos de referencia (landmarks)
    if results.pose_landmarks:
        # Dibujar los puntos de referencia en la imagen
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
        )

        # Acceder a los landmarks de las manos
        landmarks = results.pose_landmarks.landmark

        # Obtener las coordenadas de las muñecas derecha e izquierda
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

        # Convertir coordenadas normalizadas a pixeles
        h, w, _ = frame.shape
        right_wrist_x = int(right_wrist.x * w)
        right_wrist_y = int(right_wrist.y * h)
        left_wrist_x = int(left_wrist.x * w)
        left_wrist_y = int(left_wrist.y * h)

        lista_puntos_der.append([right_wrist_x,right_wrist_y])
        lista_puntos_izq.append([left_wrist_x,left_wrist_y])

        if len(lista_puntos_izq)>puntos_max:
            lista_puntos_izq.pop(0)
            lista_puntos_izq.pop(0)

        if len(lista_puntos_der)>puntos_max:
            lista_puntos_der.pop(0)
            lista_puntos_der.pop(0)

        # Dibujar cada punto en la imagen
        for punto in lista_puntos_der:
            # cv2.circle(imagen, centro, radio, color, grosor)
            cv2.circle(frame, punto, radius=5, color=(255, 255, 255), thickness=-1) 

        for punto in lista_puntos_izq:
            # cv2.circle(imagen, centro, radio, color, grosor)
            cv2.circle(frame, punto, radius=5, color=(0, 0, 0), thickness=-1) 

        # Calcular el punto medio entre ambas muñecas
        mid_x = int((right_wrist_x + left_wrist_x) / 2)
        mid_y = int((right_wrist_y + left_wrist_y) / 2)

        # Calcular la distancia entre las muñecas
        distance = np.sqrt((right_wrist_x - left_wrist_x) ** 2 + (right_wrist_y - left_wrist_y) ** 2)

        # Definir el tamaño del círculo dinámicamente
        circle_radius = max(10, int(distance / 2))  # Asegura que el radio no sea menor que 10

        # Dibujar un círculo en el punto medio
        cv2.circle(frame, (mid_x, mid_y), circle_radius, (192, 192, 192), 3)

    # Mostrar el video con las marcas dibujadas y el círculo
    cv2.imshow('MediaPipe Pose - Detección de Brazos y Manos', frame)

    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        mixer.music.stop()
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
pose.close()
